In [2]:
import pandas as pd
from geopy.distance import geodesic

# Read the CSV file
df = pd.read_csv('StationPoints.csv')
POINTS = 50

london_center = (51.5074, -0.1278)

def is_within_dist(point1, point2):
    return geodesic(point1, point2).meters < 300

def is_within_two_miles_of_london(point):
    return geodesic(point, london_center).miles <= 2

print("df loaded: ", df.shape)

filtered_points = []
for index, row in df.iterrows():
    if len(filtered_points) >= POINTS:
        break
    point = (row['Lat'], row['Lon'])
    if is_within_two_miles_of_london(point) and not any(is_within_dist(point, (p['Lat'], p['Lon'])) for p in filtered_points):
        filtered_points.append(row)

filtered_df = pd.DataFrame(filtered_points)
filtered_df.to_csv(f'stations_{POINTS}.csv', index=False)

df loaded:  (4014, 8)


In [8]:
import folium

# Create a map centered around the average latitude and longitude of the filtered points
map_center = [filtered_df['Lat'].mean(), filtered_df['Lon'].mean()]
station_map = folium.Map(location=map_center, zoom_start=12)

# Add points to the map
for index, row in filtered_df.iterrows():
    folium.Marker([row['Lat'], row['Lon']], popup=row['FriendlyName']).add_to(station_map)

# Save the map to an HTML file
station_map.save('station_map.html')


In [13]:
import json

# Load the lines data from lines.json
with open('lines.json', 'r') as file:
    lines_data = json.load(file)

# Create a map centered around the average latitude and longitude of the filtered points
map_center = [filtered_df['Lat'].mean(), filtered_df['Lon'].mean()]
line_map = folium.Map(location=map_center, zoom_start=12)

# Overlay the lines on the station map
for feature in lines_data['features']:
    points = feature['geometry']['coordinates']
    # Convert points to (lat, lon) format
    lat_lon_points = [(point[1], point[0]) for point in points]
    folium.PolyLine(lat_lon_points, color='blue', weight=2.5, opacity=1).add_to(station_map)

# Save the updated map to an HTML file
station_map.save('station_map_with_lines.html')


In [ ]:
# import tensorflow as tf
# from tensorflow.keras import layers, models

# # Define the CNN model architecture
# model = models.Sequential([
#     layers.Conv2D(32, (3, 3), activation='relu', input_shape=(height, width, num_channels)),
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(64, (3, 3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(128, (3, 3), activation='relu'),
#     layers.Flatten(),
#     layers.Dense(256, activation='relu'),
#     layers.Dense(height * width, activation='sigmoid'),  # Output layer for heat map
#     layers.Reshape((height, width))  # Reshape to heat map dimensions
# ])

# # Compile the model
# model.compile(optimizer='adam', loss='mse', metrics=['mae'])


In [ ]:
# history = model.fit(train_images, train_heatmaps, epochs=num_epochs, validation_data=(val_images, val_heatmaps))
